In [86]:
%matplotlib inline
import pandas
import numpy as np
import matplotlib.pyplot as plt
from keras import models
from keras import layers
from keras import regularizers


csv_pile_path="C:/Users/user/Desktop/딥러닝/iris.csv"
data=pandas.read_csv(csv_pile_path,sep=",") #데이터 로드
data=data.sample(frac=1).reset_index(drop=True) # 데이터를 분할했을 때, 케이스가 골고루 있게 랜덤으로 데이터 위치 섞음



#클래스 데이터타입 변환 함수(원-핫 인코딩)
def to_one_hot(labels,dimension=3):
    results=np.zeros((len(labels),dimension))
    for i,label in enumerate(labels):
        if label=="setosa":
            index=0
        elif label=="versicolor":
            index=1
        else:
            index=2
        results[i,index]=1
    return results
        
    
    
feature_data=data[["Sepal.Length","Sepal.Width","Petal.Length","Petal.Width"]] #특징 데이터만 추출
feature_data=np.array(feature_data) #행렬 변환

label_data=to_one_hot(data["Species"]) #클래스 데이터만 추출 & 벡터 변환



# 훈련 데이터와 검증 데이터, 테스트 데이터 나눔
x_train=feature_data[:120,:] 
y_train=label_data[:120,:]

x_test=feature_data[120:,:]
y_test=label_data[120:,:]



# K-겹 교차 검증
k=5
num_val_samples=len(x_train)//k

val_scores=[]
for fold in range(k):
    x_val=x_train[num_val_samples*fold:num_val_samples*(fold+1)]
    y_val=y_train[num_val_samples*fold:num_val_samples*(fold+1)]
    
    x_partial_train=np.concatenate([x_train[:num_val_samples*fold],x_train[num_val_samples*(fold+1):]],axis=0)
    y_partial_train=np.concatenate([y_train[:num_val_samples*fold],y_train[num_val_samples*(fold+1):]],axis=0)
    
    model=models.Sequential()
    model.add(layers.Dense(32,kernel_regularizer=regularizers.l2(0.001),activation='relu',input_shape=(4,)))
    model.add(layers.Dropout(0.2)) #드롭아웃으로 과대적합 완화
    model.add(layers.Dense(32,kernel_regularizer=regularizers.l2(0.001),activation='relu')) #가중치 규제로 과대적합 완화
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(3,activation='softmax')) #3개의 종류로 분류
    
    #모델 컴파일하기
    model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

    #모델 훈련
    model.fit(x_partial_train,y_partial_train,epochs=32,batch_size=8,verbose=0)
    val_mse,val_mae=model.evaluate(x_val,y_val,verbose=0)
    val_scores.append(val_mae)

#검증 점수
val_score=np.average(val_scores)
print("검증점수는 %f"%val_score) 




#튜닝한 후 테스트
model=models.Sequential()
model.add(layers.Dense(32,kernel_regularizer=regularizers.l2(0.001),activation='relu',input_shape=(4,)))
model.add(layers.Dropout(0.2)) #드롭아웃으로 과대적합 완화
model.add(layers.Dense(32,kernel_regularizer=regularizers.l2(0.001),activation='relu')) #가중치 규제로 과대적합 완화
model.add(layers.Dropout(0.2))
model.add(layers.Dense(3,activation='softmax')) #3개의 종류로 분류

model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=32,batch_size=8,verbose=0)

test_mse_score,test_acc=model.evaluate(x_test,y_test)
test_acc

model.save('iris_model.h5')

검증점수는 0.983333
30/30 [==============================] - 7s 217ms/step
